# TP1 - NER

### En utilisant vos connaissances en programmation, codez un système NER qui reconnaît les noms de personnes (PERSON) et les lieux (LOCATION) à partir de l'ensemble de données fourni (europarl).